In [0]:
!pip install pulp

Requirement already satisfied: pulp in /databricks/python3/lib/python3.8/site-packages (2.7.0)
 WARNING: You are using pip version 21.0.1; however, version 22.3.1 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

In [0]:
import pandas as pd
from pulp import LpMinimize, LpProblem, LpStatus, lpSum, LpVariable #Linear programming libraries

In [0]:
# Initialize variables
DATA_ROOT = "/dbfs/mnt/adls/40_project/SND/HACKATON/"
CH2_PATH1 = "Challenge2_capacity_private.csv"
CH2_PATH2 = "Challenge2_distances_private.csv"
CH2_PATH3 = "Challenge2_second_leg_GHG_private.csv"
CH2_PATH4 = "Challenge2_total_tires_private.csv"
CH2_PATH5 = "Challenge2_vehicles_private.csv"

# Read data into pandas dataframes
loc_cap_df = pd.read_csv(DATA_ROOT + CH2_PATH1) #the local capacity
leg1_dist_df = pd.read_csv(DATA_ROOT + CH2_PATH2)
leg2_ghg_df = pd.read_csv(DATA_ROOT + CH2_PATH3)
dc_tires_df = pd.read_csv(DATA_ROOT + CH2_PATH4)
vehicles_df = pd.read_csv(DATA_ROOT + CH2_PATH5)

In [0]:
vehicles_df

Out[40]:

,dc_name,vehicle_name,vehicle_type,base_ghg_per_mile,extra_ghg_per_tire_per_mile,max_tire_capacity
0,KANSAS CITY,DESDAMONA,EV,26,0.00405,260
1,KANSAS CITY,ELEANOR,EV,26,0.00405,260
2,ST. LOUIS,GINA,EV,26,0.00405,260
3,ST. LOUIS,BESSIE,EV,26,0.00405,260
4,SPRINGFIELD,WAGGY,EV,26,0.00405,260
5,SPRINGFIELD,ROBOCOP,EV,26,0.00405,260
6,KANSAS CITY,TERMINATOR,DIESEL,23,0.03915,335
7,KANSAS CITY,BRUM,DIESEL,23,0.03915,335
8,ST. LOUIS,BEAST,DIESEL,23,0.03915,335
9,ST. LOUIS,ARROWCAR,DIESEL,23,0.03915,335


In [0]:
# Create dictionaries containing linear program variables related to first leg GHG costs and ansialary information
vehicle_to_loc_var_list = []
for index, row in vehicles_df.iterrows():
  for index2, row2 in loc_cap_df.iterrows():
    vehicle_to_loc_var_list.append({"vehicle_name": row["vehicle_name"], \
                                    "dc_name": row["dc_name"], \
                                    "location": row2["location"], \
                                    "base_ghg_per_mile": row["base_ghg_per_mile"], \
                                    "extra_ghg_per_tire_per_mile": row["extra_ghg_per_tire_per_mile"], \
                                    "distance": leg1_dist_df[(leg1_dist_df["dc_name"] == row["dc_name"]) & (leg1_dist_df["destination"] == row2["location"])]["distance"].item(), \
                                    "leg2_base_ghg": leg2_ghg_df[(leg2_ghg_df["location"] == row2["location"])]["base_ghg"].item(), \
                                    "leg2_extra_ghg_per_tire": leg2_ghg_df[(leg2_ghg_df["location"] == row2["location"])]["additional_ghg_per_tire"].item(), \
                                    "vehicle_to_loc_var": LpVariable(name = row["vehicle_name"] + "_" + row2["location"], lowBound = 0, upBound = 1, cat = "Binary"), \
                                    "tire_per_vehicle_to_loc_var": LpVariable(name = row["vehicle_name"] + "_" + row2["location"] + "_tires", lowBound = 0, upBound = row["max_tire_capacity"], cat="Integer")})
    
# Create dictionaries containing linear program variables related to second leg GHG costs and ansialary information
tires_in_loc_var_list = []
for index, row in leg2_ghg_df.iterrows():
  if (row["location"] in dc_tires_df["dc_name"].tolist()):
    tires_in_loc_var_list.append({"location": row["location"], \
                                  "base_ghg": row["base_ghg"], \
                                  "additional_ghg_per_tire": row["additional_ghg_per_tire"], \
                                  "tires_var": LpVariable(name = row["location"] + "_tires", lowBound = 0, cat="Integer"), \
                                  "loc_var": LpVariable(name = row["location"], lowBound = 0, upBound = 1, cat = "Binary")})

Objective Function: $$ min \left [ \sum_{i=1}^{i= \rm{TruckDestinationPairs}} T_i \times \left ( (  B_1i + n_1i \times p_1i \right) \times d_i + B_3 + n_1 * p_3 ) + \sum_{j=1}^{j= \rm{SecondLegDCs}}  Q_j \times (B_2j + n_2j \times p_2j) \right ]$$

- TruckDestinationPairs:
  - \\(T\\) = 0 when truck is not used and 1 when truck is used (Variable)
  - \\(B{_1}\\) is the base GHG emitted of each truck per mile
  - \\(n{_1}\\) is the number of tires on each truck (Variable)
  - \\(p{_1}\\) is the GHG emitted per mile per tire

- SecondLegDCs:
  - \\(Q\\) = 0 when no tires are left at a location and 1 when tires are left at a location (Variable)
  - \\(B{_2}\\) is the base cost of leaving a tire at a DC
  - \\(n{_2}\\) is the number of tires left at each location (Variable)
  - \\(p{_2}\\) is the emission per tire of tires left at each location

- SecondLegNonDCs:
  - \\(B{_3}\\) is the base cost of the train and hub
  - \\(p{_3}\\) is emission per tire of tires that went to hub or train

- Other constants:
  - \\(c\\) is the maximum capacity of tires at a location
  - \\(t\\) is the initial number of scrap tires at a DC

- Constraints
  - Total tires sent to leg1 destination must be less than capacity
    - For each leg1 destination location \\(l\\): \\(\sum_{i=1}^{i=\rm{TruckDestinationPairs}} n{_1}{_l}{_i} <= c_l\\)
  - Each truck can be used a max of one time
    - For each truck \\(v\\): \\(\sum_{i=1}^{i=\rm{TruckDestinationPairs}} T{_v}{_i} <= 1\\)
  - Tires remaining at DC must be initial scrap tires - tires transported
    - For each DC \\(d\\): \\(n{\_2}{\_d} = t{\_d} - \sum_{i=1}^{i=\rm{TruckDestinationPairs}} n{\_1}{\_d}{\_i}\\)
  - If \\(n{_1} > 0\\), \\(T = 1\\); else \\(T = 0\\)
    - For each truck destination pair \\(i\\): \\(n{_1}{_i} * 1e6{^*} >= T{_i}\\)
    - For each truck destination pair \\(i\\): \\(T{_i} * 1e6{^*} >= n{_1}{_i}\\)
  - If \\(n{_2} > 0\\), \\(Q = 1\\); else \\(Q = 0\\)
    - For each DC \\(d\\): \\(n{_1}{_d} * 1e6{^*} >= Q{_d}\\)
    - For each DC \\(d\\): \\(Q{_d} * 1e6{^*} >= n{_1}{_d}\\)
    
\\({^*}\\)1,000,000 is used as a sufficiently large number to be greater than number of tires being transported

In [0]:
# Create model
model = LpProblem(name="ghg-minimize", sense=LpMinimize)

# Objective function
#   1) Loop through vehicle-destination pairs to create first leg objective function
#      - Base GHG cost per mile * distance travelled (if used)
#      - GHG cost per mile * distance travelled * tires moved
#   2) Loop through destination locations to create second leg objective function
#      - Base GHG cost
#      - GHG per tire * number of tires
obj_func = 0
for vehicle_loc in vehicle_to_loc_var_list:
  obj_func = obj_func + vehicle_loc["vehicle_to_loc_var"] * (vehicle_loc["base_ghg_per_mile"] * vehicle_loc["distance"] + vehicle_loc["leg2_base_ghg"]) + \
        vehicle_loc["tire_per_vehicle_to_loc_var"] * (vehicle_loc["extra_ghg_per_tire_per_mile"] * vehicle_loc["distance"] + vehicle_loc["leg2_extra_ghg_per_tire"])
for tires_in_loc in tires_in_loc_var_list:
  obj_func = obj_func + tires_in_loc["loc_var"] * tires_in_loc["base_ghg"] + \
        tires_in_loc["tires_var"] * tires_in_loc["additional_ghg_per_tire"]
model += obj_func

# Location constraints
#   Loop through vehicle-destination pairs
#     - If tires are transported, the vehicle-destination pair is flagged as used (1); otherwise it is flagged as not used (0)
#     - Sum of tires transported to destination must be less than or equal to max capacity
loc_constraints = {}
for vehicle_loc in vehicle_to_loc_var_list:
  if vehicle_loc["location"] in loc_constraints:
    loc_constraints[vehicle_loc["location"]] = loc_constraints[vehicle_loc["location"]] + vehicle_loc["tire_per_vehicle_to_loc_var"]
  else:
    loc_constraints[vehicle_loc["location"]] = vehicle_loc["tire_per_vehicle_to_loc_var"]
  model += (vehicle_loc["vehicle_to_loc_var"] * 1e6 >= vehicle_loc["tire_per_vehicle_to_loc_var"], vehicle_loc["location"] + "_" + vehicle_loc["vehicle_name"] + "_boolean_constraint_1")
  model += (vehicle_loc["tire_per_vehicle_to_loc_var"] * 1e6 >= vehicle_loc["vehicle_to_loc_var"], vehicle_loc["location"] + "_" + vehicle_loc["vehicle_name"] + "_boolean_constraint_2")
for key in loc_constraints:
  model += (loc_constraints[key] <= loc_cap_df[loc_cap_df["location"] == key]["max_tire_capacity"].item(), key + "_max_constraint")
  
# Vehicle constraints
#   Loop through vehicles
#     - Sum of vehicle use (1 or 0) must be less than or equal to 1
vehicle_constraints = {}
for vehicle in vehicle_to_loc_var_list:
  if vehicle["vehicle_name"] in vehicle_constraints:
    vehicle_constraints[vehicle["vehicle_name"]] = vehicle_constraints[vehicle["vehicle_name"]] + vehicle["vehicle_to_loc_var"]
  else:
    vehicle_constraints[vehicle["vehicle_name"]] = vehicle["vehicle_to_loc_var"]
for key in vehicle_constraints:
  model += (vehicle_constraints[key] <= 1, key + "_max_constraint")

# Total tires at location must equal sum of tires delivered
#   Loop through locations
#     - Hub, Train, and Recycle are always used (= 1)
#     - For Hub, Train, and Recycle tires at location must equal sum of tires transported to location
#     - For Non-hub, train, and recycle tires at location, must equal initial tires - sum of tires shipped
#     - If tires remain present in a dc, set that dc as used (1); otherwise flag it as not used (0)
dc_constraints = {}
for vehicle_loc in vehicle_to_loc_var_list:
  if vehicle_loc["dc_name"] in dc_constraints:
    dc_constraints[vehicle_loc["dc_name"]] = dc_constraints[vehicle_loc["dc_name"]] - vehicle_loc["tire_per_vehicle_to_loc_var"]
  else:
    dc_constraints[vehicle_loc["dc_name"]] = -vehicle_loc["tire_per_vehicle_to_loc_var"]
for loc in tires_in_loc_var_list:
  if loc["location"] in loc_constraints:
    model += (loc_constraints[loc["location"]] - loc["tires_var"] == 0, loc["location"] + "_tot_constraint")
    model += (loc["loc_var"] == 1, loc["location"] + "_bool")
  if loc["location"] in dc_constraints:
    model += (dc_tires_df[dc_tires_df["dc_name"] == loc["location"]]["tires"].item() + dc_constraints[loc["location"]] == loc["tires_var"], loc["location"] + "_tot_constraint")
  model += (loc["tires_var"] * 1e6 >= loc["loc_var"], loc["location"] + "_boolean_constraint_1")
  model += (loc["loc_var"] * 1e6 >= loc["tires_var"], loc["location"] + "_boolean_constraint_2")

In [0]:
model

Out[27]: ghg-minimize:
MINIMIZE
399600*ANDROMEDA_HUB + 118.91*ANDROMEDA_HUB_tires + 37206*ANDROMEDA_RECYCLER + 5.7955499999999995*ANDROMEDA_RECYCLER_tires + 227848*ANDROMEDA_TRAIN + 67.1594*ANDROMEDA_TRAIN_tires + 389704*ARROWCAR_HUB + 126.1892*ARROWCAR_HUB_tires + 21459*ARROWCAR_RECYCLER + 36.52695*ARROWCAR_RECYCLER_tires + 224690*ARROWCAR_TRAIN + 67.7345*ARROWCAR_TRAIN_tires + 389704*BEAST_HUB + 126.1892*BEAST_HUB_tires + 21459*BEAST_RECYCLER + 36.52695*BEAST_RECYCLER_tires + 224690*BEAST_TRAIN + 67.7345*BEAST_TRAIN_tires + 390448*BESSIE_HUB + 117.48440000000001*BESSIE_HUB_tires + 24258*BESSIE_RECYCLER + 3.77865*BESSIE_RECYCLER_tires + 224780*BESSIE_TRAIN + 66.6815*BESSIE_TRAIN_tires + 384115*BRUM_HUB + 116.67575000000001*BRUM_HUB_tires + 17457*BRUM_RECYCLER + 29.71485*BRUM_RECYCLER_tires + 224506*BRUM_TRAIN + 67.4213*BRUM_TRAIN_tires + 384130*DESDAMONA_HUB + 116.50025000000001*DESDAMONA_HUB_tires + 19734*DESDAMONA_RECYCLER + 3.07395*DESDAMONA_RECYCLER_tires + 224572*DESDAMONA_TRAIN + 66.6491*DESDAMONA_TRAIN_tires + 384130*ELEANOR_HUB + 116.50025000000001*ELEANOR_HUB_tires + 19734*ELEANOR_RECYCLER + 3.07395*ELEANOR_RECYCLER_tires + 224572*ELEANOR_TRAIN + 66.6491*ELEANOR_TRAIN_tires + 395544*EVEREST_HUB + 118.2782*EVEREST_HUB_tires + 30446*EVEREST_RECYCLER + 4.74255*EVEREST_RECYCLER_tires + 226080*EVEREST_TRAIN + 66.884*EVEREST_TRAIN_tires + 1396750*FARGO + 388.6305*FARGO_tires + 394212*GALAXY_HUB + 133.86260000000001*GALAXY_HUB_tires + 26933*GALAXY_RECYCLER + 45.844649999999994*GALAXY_RECYCLER_tires + 225840*GALAXY_TRAIN + 69.69200000000001*GALAXY_TRAIN_tires + 390448*GINA_HUB + 117.48440000000001*GINA_HUB_tires + 24258*GINA_RECYCLER + 3.77865*GINA_RECYCLER_tires + 224780*GINA_TRAIN + 66.6815*GINA_TRAIN_tires + 395544*K2_HUB + 118.2782*K2_HUB_tires + 30446*K2_RECYCLER + 4.74255*K2_RECYCLER_tires + 226080*K2_TRAIN + 66.884*K2_TRAIN_tires + 1016640*KANSAS_CITY + 252.5952*KANSAS_CITY_tires + 397800*LIGHTY_HUB + 139.97*LIGHTY_HUB_tires + 32913*LIGHTY_RECYCLER + 56.023649999999996*LIGHTY_RECYCLER_tires + 227404*LIGHTY_TRAIN + 72.3542*LIGHTY_TRAIN_tires + 1324650*MINNEAPOLIS + 356.8095*MINNEAPOLIS_tires + 399600*PULSAR_HUB + 118.91*PULSAR_HUB_tires + 37206*PULSAR_RECYCLER + 5.7955499999999995*PULSAR_RECYCLER_tires + 227848*PULSAR_TRAIN + 67.1594*PULSAR_TRAIN_tires + 388238*ROBOCOP_HUB + 117.14015*ROBOCOP_HUB_tires + 22594*ROBOCOP_RECYCLER + 3.51945*ROBOCOP_RECYCLER_tires + 225300*ROBOCOP_TRAIN + 66.7625*ROBOCOP_TRAIN_tires + 394212*ROCKET_HUB + 133.86260000000001*ROCKET_HUB_tires + 26933*ROCKET_RECYCLER + 45.844649999999994*ROCKET_RECYCLER_tires + 225840*ROCKET_TRAIN + 69.69200000000001*ROCKET_TRAIN_tires + 387749*ROCKY_HUB + 122.86145*ROCKY_HUB_tires + 19987*ROCKY_RECYCLER + 34.02135*ROCKY_RECYCLER_tires + 225150*ROCKY_TRAIN + 68.5175*ROCKY_TRAIN_tires + 1122240*SPRINGFIELD + 289.2032*SPRINGFIELD_tires + 1183680*ST._LOUIS + 310.5024*ST._LOUIS_tires + 384115*TERMINATOR_HUB + 116.67575000000001*TERMINATOR_HUB_tires + 17457*TERMINATOR_RECYCLER + 29.71485*TERMINATOR_RECYCLER_tires + 224506*TERMINATOR_TRAIN + 67.4213*TERMINATOR_TRAIN_tires + 387749*VIPER_HUB + 122.86145*VIPER_HUB_tires + 19987*VIPER_RECYCLER + 34.02135*VIPER_RECYCLER_tires + 225150*VIPER_TRAIN + 68.5175*VIPER_TRAIN_tires + 388238*WAGGY_HUB + 117.14015*WAGGY_HUB_tires + 22594*WAGGY_RECYCLER + 3.51945*WAGGY_RECYCLER_tires + 225300*WAGGY_TRAIN + 66.7625*WAGGY_TRAIN_tires + 397800*ZOOM_HUB + 139.97*ZOOM_HUB_tires + 32913*ZOOM_RECYCLER + 56.023649999999996*ZOOM_RECYCLER_tires + 227404*ZOOM_TRAIN + 72.3542*ZOOM_TRAIN_tires + 0.0
SUBJECT TO
HUB_DESDAMONA_boolean_constraint_1: 1000000 DESDAMONA_HUB
 - DESDAMONA_HUB_tires >= 0

HUB_DESDAMONA_boolean_constraint_2: - DESDAMONA_HUB
 + 1000000 DESDAMONA_HUB_tires >= 0

TRAIN_DESDAMONA_boolean_constraint_1: 1000000 DESDAMONA_TRAIN
 - DESDAMONA_TRAIN_tires >= 0

TRAIN_DESDAMONA_boolean_constraint_2: - DESDAMONA_TRAIN
 + 1000000 DESDAMONA_TRAIN_tires >= 0

RECYCLER_DESDAMONA_boolean_constraint_1: 1000000 DESDAMONA_RECYCLER
 - DESDAMO

In [0]:
status = model.solve()

In [0]:
model.objective.value()

Out[29]: 2674911.8374999994

In [0]:
for var in model.variables():
  print(f"{var.name}: {var.value()}")

ANDROMEDA_HUB: 0.0
ANDROMEDA_HUB_tires: 0.0
ANDROMEDA_RECYCLER: 1.0
ANDROMEDA_RECYCLER_tires: 192.0
ANDROMEDA_TRAIN: 0.0
ANDROMEDA_TRAIN_tires: 0.0
ARROWCAR_HUB: 0.0
ARROWCAR_HUB_tires: 0.0
ARROWCAR_RECYCLER: 1.0
ARROWCAR_RECYCLER_tires: 329.0
ARROWCAR_TRAIN: 0.0
ARROWCAR_TRAIN_tires: 0.0
BEAST_HUB: 0.0
BEAST_HUB_tires: 0.0
BEAST_RECYCLER: 1.0
BEAST_RECYCLER_tires: 335.0
BEAST_TRAIN: 0.0
BEAST_TRAIN_tires: 0.0
BESSIE_HUB: 0.0
BESSIE_HUB_tires: 0.0
BESSIE_RECYCLER: 1.0
BESSIE_RECYCLER_tires: 260.0
BESSIE_TRAIN: 0.0
BESSIE_TRAIN_tires: 0.0
BRUM_HUB: 0.0
BRUM_HUB_tires: 0.0
BRUM_RECYCLER: 0.0
BRUM_RECYCLER_tires: 0.0
BRUM_TRAIN: 0.0
BRUM_TRAIN_tires: 0.0
DESDAMONA_HUB: 0.0
DESDAMONA_HUB_tires: 0.0
DESDAMONA_RECYCLER: 0.0
DESDAMONA_RECYCLER_tires: 0.0
DESDAMONA_TRAIN: 0.0
DESDAMONA_TRAIN_tires: 0.0
ELEANOR_HUB: 0.0
ELEANOR_HUB_tires: 0.0
ELEANOR_RECYCLER: 0.0
ELEANOR_RECYCLER_tires: 0.0
ELEANOR_TRAIN: 0.0
ELEANOR_TRAIN_tires: 0.0
EVEREST_HUB: 0.0
EVEREST_HUB_tires: 0.0
EVEREST_RECYCLER: 0.0
EVEREST_RECYCLER_tires: 0.0
EVEREST_TRAIN: 0.0
EVEREST_TRAIN_tires: 0.0
FARGO: 0.0
FARGO_tires: 0.0
GALAXY_HUB: 0.0
GALAXY_HUB_tires: 0.0
GALAXY_RECYCLER: 1.0
GALAXY_RECYCLER_tires: 297.0
GALAXY_TRAIN: 0.0
GALAXY_TRAIN_tires: 0.0
GINA_HUB: 0.0
GINA_HUB_tires: 0.0
GINA_RECYCLER: 0.0
GINA_RECYCLER_tires: 0.0
GINA_TRAIN: 0.0
GINA_TRAIN_tires: 0.0
K2_HUB: 0.0
K2_HUB_tires: 0.0
K2_RECYCLER: 0.0
K2_RECYCLER_tires: 0.0
K2_TRAIN: 0.0
K2_TRAIN_tires: 0.0
KANSAS_CITY: 1.0
KANSAS_CITY_tires: 999.0
LIGHTY_HUB: 0.0
LIGHTY_HUB_tires: 0.0
LIGHTY_RECYCLER: 0.0
LIGHTY_RECYCLER_tires: 0.0
LIGHTY_TRAIN: 1.0
LIGHTY_TRAIN_tires: 335.0
MINNEAPOLIS: 0.0
MINNEAPOLIS_tires: 0.0
PULSAR_HUB: 0.0
PULSAR_HUB_tires: 0.0
PULSAR_RECYCLER: 0.0
PULSAR_RECYCLER_tires: 0.0
PULSAR_TRAIN: 0.0
PULSAR_TRAIN_tires: 0.0
ROBOCOP_HUB: 0.0
ROBOCOP_HUB_tires: 0.0
ROBOCOP_RECYCLER: 1.0
ROBOCOP_RECYCLER_tires: 260.0
ROBOCOP_TRAIN: 0.0
ROBOCOP_TRAIN_tires: 0.0
ROCKET_HUB: 0.0
ROCKET_HUB_tires: 0.0
ROCKET_RECYCLER: 0.0
ROCKET_RECYCLER_tires: 0.0
ROCKET_TRAIN: 1.0
ROCKET_TRAIN_tires: 335.0
ROCKY_HUB: 0.0
ROCKY_HUB_tires: 0.0
ROCKY_RECYCLER: 1.0
ROCKY_RECYCLER_tires: 327.0
ROCKY_TRAIN: 0.0
ROCKY_TRAIN_tires: 0.0
SPRINGFIELD: 0.0
SPRINGFIELD_tires: 0.0
ST._LOUIS: 0.0
ST._LOUIS_tires: 0.0
TERMINATOR_HUB: 0.0
TERMINATOR_HUB_tires: 0.0
TERMINATOR_RECYCLER: 0.0
TERMINATOR_RECYCLER_tires: 0.0
TERMINATOR_TRAIN: 0.0
TERMINATOR_TRAIN_tires: 0.0
VIPER_HUB: 1.0
VIPER_HUB_tires: 324.0
VIPER_RECYCLER: 0.0
VIPER_RECYCLER_tires: 0.0
VIPER_TRAIN: 0.0
VIPER_TRAIN_tires: 0.0
WAGGY_HUB: 0.0
WAGGY_HUB_tires: 0.0
WAGGY_RECYCLER: 0.0
WAGGY_RECYCLER_tires: 0.0
WAGGY_TRAIN: 0.0
WAGGY_TRAIN_tires: 0.0
ZOOM_HUB: 0.0
ZOOM_HUB_tires: 0.0
ZOOM_RECYCLER: 0.0
ZOOM_RECYCLER_tires: 0.0
ZOOM_TRAIN: 1.0
ZOOM_TRAIN_tires: 335.0

In [0]:
import numpy as np
Trucks = [] #list to store truck name
DC = [] #list to store distribution center of each truck
Dest = [] #list to store destination of each truck
Tires = []#list to store the number of tires on each truck
Veh_type = []
Dist = []

for var in model.variables(): #loop through each variable of model
  if('_tires' in var.name): #the lines with '_tires' are the ones with the number of tires on each truck
    if(var.value()>0): #if value > 0 then there are tires on truck
      split_str = var.name.split('_') #split this string to get different variables
      if((len(split_str)==3) & (split_str[0]!='KANSAS')): #if -t = 3 splits, it is a truck if it was 2 that would be the number of tires from each hub
        Trucks.append(split_str[0]) #append truck name to list
        #Veh_type.append(vehicles_df['vehicle_type'][split_str[0] == vehicles_df['vehicle_name']].item())
        Dest.append(split_str[1]) #append destination to list
        Tires.append(var.value()) #append the number of tires to list
        DC.append(vehicles_df['dc_name'][split_str[0] == vehicles_df['vehicle_name']].item()) #append dc that truck left from
        Dist.append(leg1_dist_df['distance'][(vehicles_df['dc_name'][split_str[0] == vehicles_df['vehicle_name']].item() == leg1_dist_df['dc_name']) & (split_str[1] == leg1_dist_df['destination'])].item())

  

In [0]:
#save these lists to a df
submission_df = pd.DataFrame(list(zip(DC, Trucks, Dest,Tires)),
               columns =['dc_name', 'vehicle_name','destination','number_of_tires_shipped'])

submission_df.to_csv(DATA_ROOT + '/DATA_OUT/opt_submit_final.csv')
submission_df

Out[41]:

,dc_name,vehicle_name,destination,number_of_tires_shipped
0,FARGO,ANDROMEDA,RECYCLER,192.0
1,ST. LOUIS,ARROWCAR,RECYCLER,329.0
2,ST. LOUIS,BEAST,RECYCLER,335.0
3,ST. LOUIS,BESSIE,RECYCLER,260.0
4,MINNEAPOLIS,GALAXY,RECYCLER,297.0
5,FARGO,LIGHTY,TRAIN,335.0
6,SPRINGFIELD,ROBOCOP,RECYCLER,260.0
7,MINNEAPOLIS,ROCKET,TRAIN,335.0
8,SPRINGFIELD,ROCKY,RECYCLER,327.0
9,SPRINGFIELD,VIPER,HUB,324.0


In [0]:
#THIS CELL Validates the solution
import requests
url = "https://scoring-app-uuzeqpiufa-ue.a.run.app/optimization2/validate"
payload = {
"team_key": "nQDwNYY8",
"data": submission_df.to_dict(orient="records")
}
response = requests.post(url, json=payload)
print(response.status_code, response.content)

200 b'{"message":"Success"}'

In [0]:
#THIS CELL SUBMITS the solution
import requests
url = "https://scoring-app-uuzeqpiufa-ue.a.run.app/optimization2/submit"
payload = {
"team_key": "nQDwNYY8",
"data": submission_df.to_dict(orient="records")
}
#response = requests.post(url, json=payload)
#print(response.status_code, response.content)

201 b'{"message":"Success","challenge":"optimization"}'